# Assignment 1 - MATH60837A
William Bourque - 11359215

Frederic Pelletier - 11359258

## installing dependencies (only needed once)

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install pandas
%pip install statsmodels

## importing dependencies

In [67]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from statsmodels.tsa.stattools import acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import chi2
import warnings
warnings.filterwarnings("ignore")


## Loading the data

In [ ]:
df = pd.read_csv("./data.csv")
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
print(df.head())

## Part 1 - Preliminary Analysis

### 1. a) Plot the log of house prices through time. What does a change on y-axis i.e. log(HP_t) - log(HP_{t-1}) represent?

In [ ]:
df['log_HP'] = np.log(df['QCAR628BIS'])
sns.lineplot(data=df, x='DATE', y='log_HP')
plt.ylabel('Log of house prices')
plt.title("log(HP_t)")
plt.show()

### Interpretation:
log(HP_t) - log(HP_{t-1}) represents the percentage of gain (if positive) or loss (if negative) on house prices from time {t-1} to time {t}

### 1. b) Assume there is a deterministic linear time trend. Apply the transformation to stationarize z_t and plot the results (determinist trend)

In [ ]:
T = df.shape[0]  # Number of observations
trend = np.arange(1, T + 1)  # Linear time trend 

X = np.column_stack((np.ones(T), trend))  # Column of ones for intercept, trend as second column

# Dependent variable
Y = np.array(df['log_HP'])  # Assuming log_HP is the dependent variable in the DataFrame

# Solve for B using the OLS formula
B = np.linalg.lstsq(X, Y, rcond=None)[0]  #OLS solution

# Calculate the detrended series
df['detrend_HP'] = Y - np.dot(X, B)  

print("Beta coefficients (B):", B)
print(df.head())

# Plot the graph of the determinist trend deviation
sns.lineplot(data=df, x='DATE', y='detrend_HP')
plt.ylabel('Growth percentage')
plt.title('Deterministic Trend Deviation')
plt.show()

### Interpretation
We can see from the graph that the deterministic trend does not seem to return a stationnarized series or to be white noise.

### 1.c) Assume there is a stochastic time trend, i.e. a random walk. Apply the transformation required to stationarize z_t and plot the resulting series.

In [ ]:
df['diff_HP'] = df['log_HP'].pct_change() # Create a column of the pct change between each periods

print(df.head())

# Plot of the log avariation assuming a stochastiv trend
sns.lineplot(data=df, x='DATE', y='diff_HP')
plt.ylabel('Growth Percentage')
plt.title("Difference Series")
plt.show()

### Interpretation
The stochastic trend series seems stationnary, there is also a possibility to be in presence of white noise. Further testing will be required to confirm or infirm both of these takes.

### 1.d) Analyze the sample autocorrelation functions to evaluate if the series are stationary. Which series would you choose to estimate a time-series model?

In [ ]:
acf_vector_deter = acf(df['detrend_HP'], nlags=20) # Autocorrelation determinist
print(acf_vector_deter)
acf_vector_stocha = acf(df['diff_HP'].iloc[1:], nlags=20) # Autocorrelation stochastic
print(acf_vector_stocha)

plt.plot(acf_vector_deter, label='deterministic acf') # Plot of determinist autocorrelations
plt.plot(acf_vector_stocha, label='stochastic acf') # Plot of stochastic autocorrelations
plt.xlabel('lag')
plt.ylabel('autocorrelation')
plt.title('ACF for deterministic vs stochastic model')
plt.legend()
plt.show()

### Interpretation
 Stochastic hypothesis is better to estimate a time series model, the autocorrelations indicates stationarity and that there could be white noise. White noise and stationarity hypothesis is refused in the case of the deterministic linear time trend.

### 1. e) Perform Lyung-Box test on the stochastic trend series

In [ ]:
lags = range(1, 19)  # Creat a vector for lags 1 to 18

# Perform the Ljung-Box Q-test
lbq_result = acorr_ljungbox(df['diff_HP'].iloc[1:], lags=lags, return_df=True)

p_values = lbq_result['lb_pvalue']
test_statistics = lbq_result['lb_stat']
h1 = (p_values < 0.05).astype(int)  # Binary decision rule (1 = reject null hypothesis)

# Print the results
print("Decision Rule (h1):", h1.values)
print("P-Values:", p_values.values)
print("Test Statistics:", test_statistics.values)

### Interpretation 
We reject the hypothesis that there is no autocorrelation between the observations of the stochastic trend time series. We can conclude that the series is not white noise. Althought it is not white noise, the autocorrelations of the stochastic trend time series suggests stationarity.

## Part 2 - Model selection

### 2.a) Estimate all 4 models by maximum likelihood, report the estimation results, and verify if the stationarity conditions are satisfied

In [74]:
yt = df['diff_HP'].dropna()

### Model 1

In [ ]:
# this is a regular AR(1) model so we can use the statsmodel library
model1 = ARIMA(yt, order=(1,0,0)) 
fitted_model1 = model1.fit()

phi1 = fitted_model1.arparams[0]
delta = fitted_model1.params['const']
variance = fitted_model1.params['sigma2']


print(f'fitter parameters: (phi1, {phi1}), (delta, {delta}), (sigma,{np.sqrt(variance)})')
if abs(phi1) < 1:
    print(f'Since |{phi1}| < 1, the model is stationary')
else:
    print(f'Since |{phi1}| > 1, the model is not stationary')
print(fitted_model1.summary())


### Interpretation
Given that the phi1 parameter is less than 1 in absolute value, the model is stationary. Furthermore, the p-values obtained from the t-tests on the parameters is less than 0.05, meaning we can reject the null hypothesis that the parameters are equal to 0. There is therefore statistically significant evidence that the parameters contribute meaningfully to the model.

### Model 2

In [ ]:
# this is a regular AR(2) model so we can use the statsmodel library
model2 = ARIMA(yt, order=(2,0,0)) 
fitted_model2 = model2.fit()

phi1 = fitted_model2.arparams[0]
phi2 = fitted_model2.arparams[1]
delta = fitted_model2.params['const']
variance = fitted_model2.params['sigma2']


print(f'fitter parameters: (phi1, {phi1}), (phi2, {phi2}) (delta, {delta}), (sigma,{np.sqrt(variance)})')
print(fitted_model2.summary())
phi_matrix = np.array([[phi1,phi2],[1,0]])
eigenvalues,_ = np.linalg.eig(phi_matrix)
print(f'The eigenvalues are {eigenvalues}')
for eigenvalue in eigenvalues:
    print(f'module of {eigenvalue} is {np.abs(eigenvalue)}')


### Interpretation
Given that the module of the eigenvalues are less than 1, we can conclude that the process is stationary. Furthermore, since the p-value of the various t-tests done on the parameters are less than 0.01, there is strong statistical evidence that we can reject the null hypothesis, meaning the parameters contribute to the model

### Model 3

In [ ]:
# this is a regular AR(3) model so we can use the statsmodel library
model3 = ARIMA(yt, order=(3,0,0))
fitted_model3 = model3.fit()

phi1 = fitted_model3.arparams[0]
phi2 = fitted_model3.arparams[1]
phi3 = fitted_model3.arparams[2]
delta = fitted_model3.params['const']
variance = fitted_model3.params['sigma2']


print(f'fitter parameters: (phi1, {phi1}), (phi2, {phi2}), (phi3, {phi3}) (delta, {delta}), (sigma,{np.sqrt(variance)})')
print(fitted_model3.summary())
phi_matrix = np.array([[phi1,phi2,phi3],[1,0,0], [0,1,0]])
eigenvalues,_ = np.linalg.eig(phi_matrix)
print(f'The eigenvalues are {eigenvalues}')
for eigenvalue in eigenvalues:
    print(f'module of {eigenvalue} is {np.abs(eigenvalue)}')



### Interpretation
Given that the module of the eigenvalues are less than 1, we can conclude that the process is stationary. Furthermore, since the p-value of the various t-tests done on the parameters are less than 0.05, there is strong statistical evidence that we can reject the null hypothesis, meaning the parameters contribute to the model meaningfully.

### Model 4

In [ ]:
# this is an AR(3) model but with a fixed value of 0 for phi2
model4 = ARIMA(yt, order=(3,0,0), enforce_stationarity=False) #needed for us to be able to fix a parameter
with model4.fix_params({'ar.L2': 0}):
    fitted_model4 = model4.fit()

phi1 = fitted_model4.arparams[0]
phi2 = fitted_model4.arparams[1]
phi3 = fitted_model4.arparams[2]
delta = fitted_model4.params['const']
variance = fitted_model4.params['sigma2']


print(f'fitter parameters: (phi1, {phi1}), (phi2, {phi2}), (phi3, {phi3}) (delta, {delta}), (sigma,{np.sqrt(variance)})')
print(fitted_model4.summary())
phi_matrix = np.array([[phi1,phi2,phi3],[1,0,0], [0,1,0]])
eigenvalues,_ = np.linalg.eig(phi_matrix)
print(f'The eigenvalues are {eigenvalues}')
for eigenvalue in eigenvalues:
    print(f'module of {eigenvalue} is {np.abs(eigenvalue)}')

### Interpretation
Given that the module of the eigenvalues are less than 1, we can conclude that the process is stationary. However, the p-value for the constant parameter is > 0.05, which means there is weak statistical evidence that this parameter contributes meaningfully to the model. The other parameters seem to contribute meaningfully.

### 2.b) Perform 2 likelihood ratio tests to justify the selection of models. The first test should discriminate between model (1) and (2), the second one should discriminate between model (3) and (4). Finally, choose one of the two remaining models using the BIC criterion

In [79]:
# we first define a function to help us perform the test
def likelihood_ratio_test(l1,l0,k, alpha):
    statistic = 2*(l1 - l0)
    critical_value = chi2.ppf(1 - alpha, k)
    print(f'{statistic=}, {critical_value=}')
    print('p-value:',1 - chi2.cdf(statistic, k)) # P(chi2 > statistic)
    return statistic > critical_value


In [ ]:

model_from_first_test = None
model_from_second_test = None
can_reject_null = likelihood_ratio_test(fitted_model2.llf, fitted_model1.llf, 1, 0.05)
if can_reject_null:
    print('we can reject the null hypothesis, meaning we pick the model 2')
    model_from_first_test = fitted_model2
else:
    print('we cannot reject the null hypothesis, meaning we pick the model 1')
    model_from_first_test = fitted_model1

can_reject_null = likelihood_ratio_test(fitted_model3.llf, fitted_model4.llf, 1, 0.05)
if can_reject_null:
    print('we can reject the null hypothesis, meaning we pick the model 3')
    model_from_second_test = fitted_model3
else:
    print('we cannot reject the null hypothesis, meaning we pick the model 4')
    model_from_second_test = fitted_model4

print(f'BIC values of selected models:{model_from_first_test.bic,model_from_second_test.bic}')
chosen_model = model_from_first_test if model_from_first_test.bic < model_from_second_test.bic else model_from_second_test

### Interpretation 
Since the BIC value is smaller for model 3, we pick it over model 2.

### 2.c) Evaluate the white noise hypothesis for the residual of the chosen model. What can you conclude?

In [ ]:
# we perform Ljung-Box on the residuals
residuals = chosen_model.resid
lags = range(1, 19)  # Creat a vector for lags 1 to 18

# Perform the Ljung-Box Q-test
lbq_result = acorr_ljungbox(residuals, lags=lags, return_df=True)

p_values = lbq_result['lb_pvalue']
test_statistics = lbq_result['lb_stat']
h1 = (p_values < 0.05).astype(int)  # Binary decision rule (1 = reject null hypothesis)

# Print the results
print("Decision Rule (h1):", h1.values)
print("P-Values:", p_values.values)
print("Test Statistics:", test_statistics.values)
acf(residuals,nlags=20)


### Interpretation
We can reject the null hypothesis for all the lags except the first 3. This indicates that the residuals are autocorrelated as we increased the lag and are therefore not white noise. Our model is therefore a poor fit for the data that we have.

## Part 3 - Dynamic response and forecasting

### 3.a) For the chosen model evaluate the dynamic response for an horizon of 10 periods following a negative shock of size sigma = 1.15 occuring at the first period of the horizon.

In [ ]:
plt.plot(chosen_model.impulse_responses(10,[-1.15]), label='response')
plt.xlabel('Periods')
plt.ylabel('Response value')
plt.title('Dynamic response following a shock of -1.5')
plt.legend()

### Interpretation
We can see the response trending to 0 which means the shock is transitory. This can be explained by the fact that the series is stationary. Even though we do not have white noise, the derivative Y_{t+i} given U_t remains the same as a regular AR(3). 

### 3.b) Forecasting: Split the sample into a training sample and a holdout sample. The holdout sample should consist of the last 34 observations.

In [ ]:
training_set = yt[:-34]
test_set = yt[-34:] #last 34 data points
model3_train = ARIMA(training_set,order=(3,0,0))
fitted_model3_train = model3_train.fit()


# y{t+5} = delta + phi1 * y{t+4} + phi2 * y{t+3} + phi3 * y{t+2}
# y{t+4} = delta + phi1 * y{t+3} + phi2 * y{t+2} + phi3 * y{t+1}
# y{t+3} = delta + phi1 * y{t+2} + phi2 * y{t+1} + phi3 * y{t}
# forecast of y{t+5} given y{t+2}, y{t+1}, y{t}

phi1 = fitted_model3_train.arparams[0]
phi2 = fitted_model3_train.arparams[1]
phi3 = fitted_model3_train.arparams[2]
delta = fitted_model3_train.params['const']
forecasts = []

for i in range(34 - 2):
    y_t3 = delta + phi1 * test_set.iloc[i+2] + phi2 * test_set.iloc[i+1] + phi3 * test_set.iloc[i]
    y_t4 = delta + phi1 * y_t3 + phi2 * test_set.iloc[i+2] + phi3 * test_set.iloc[i+1] 
    y_t5 = delta + phi1 * y_t4 + phi2 * y_t3 + phi3 * test_set.iloc[i+2]
    forecasts.append(y_t5)
forecasts_df = pd.DataFrame({'forecast': forecasts})
forecasts_df.index = test_set.index[2:]
print(test_set.mean())
plt.plot(forecasts_df, label='forecast')
plt.plot(test_set, label='true value')
plt.xlabel('DATE')
plt.ylabel('Growth percentage')
plt.title('Growth percentage over time of House prices: forecast vs. true value')
plt.legend()

### Interpretation
The forecast oscillates around the mean which is approximately 0.0017, which is expected, and it captures some of the volatility of the data, however without much success.

### 3.c) Calculate two criteria of forecast quality: the Mean Errors and the Mean Absolute Errors associated with the forecasts of step 3.b). Which of the two criteria is larger and why?

In [ ]:
mean_error = (forecasts - test_set[2:]).mean()
abs_mean_error = (forecasts - test_set[2:]).abs().mean()
print(f'mean error: {mean_error}, mean absolute error: {abs_mean_error}')

### Interpretation
The mean absolute error is bigger (an order of magnitute larger) since the positive and negative errors will cancel out in the mean error computation, resulting in a potentially lower value.